In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
import imageio

In [2]:
#F=r/(r^2+eps)^(3/2)
#u=r^2+eps, du=2rdr
#=> F=du/2u^3/2, Pot=2u^-1/2=-(r^2+eps)^-1/2

def get_forces(xy,f,soft=0.01):
    n=xy.shape[0]
    tot=0
    for i in range(n):
        dx=xy[:,0]-xy[i,0]
        dy=xy[:,1]-xy[i,1]
        rsqr=dx**2+dy**2+soft**2
        rrt=rsqr**-0.5
        r3=rrt**3
        f[i,0]=np.sum(dx*r3)
        f[i,1]=np.sum(dy*r3)
        #if pot:
        tot=tot+np.sum(rrt)
    #if pot:
        return f, tot


In [3]:
class particles:
    def __init__(self,n,soft=0.01):
        self.n=n
        self.xy=np.random.randn(n,2)
        self.v=np.zeros([n,2])
        self.f=np.zeros([n,2])
        self.soft=soft
        
    #def get_forces(self):
     #   pot=get_forces(self.xy,self.f,self.soft,pot=True)
      #  return pot
    
    def update(self,dt):
        #k is for x, dx/dt
        #m is for v, dv/dt
        #p is for potential
        k1 = self.v
        m1, p1 = get_forces(self.xy, self.f, self.soft)
        
        k2 = self.v + m1*dt/2
        m2, p2 = get_forces(self.xy + k1*dt/2, self.f, self.soft)
        
        k3= self.v + m2*dt/2
        m3, p3 = get_forces(self.xy + k2*dt/2, self.f, self.soft)
        
        k4= self.v + m3*dt
        m4, p4= get_forces(self.xy + k3*dt, self.f, self.soft)  
        
        tot_k=(k1 + 2*k2+2*k3 + k4)/6
        tot_m=(m1 + 2*m2+2*m3 + m4)/6
        tot_p=(p1 + 2*p2+2*p3 + p4)/6
        
        self.xy=self.xy + tot_k*dt
        self.v=self.v + tot_m*dt
        
        kin = np.sum(self.v**2)
        pot = tot_p

        
        return kin,-1 *pot

In [4]:
n = 1000
parts=particles(n)
dt=3e-4
plt.ion()
nstep=200
#nstep= 500
kk=np.zeros(nstep)
pp=np.zeros(nstep)

a=open('energy_RK4.txt','w')
filenames = []
for i in range(nstep):
    plt.clf()
    plt.plot(parts.xy[:,0],parts.xy[:,1],'.')
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()
    #plt.pause(0.001)
    
    kk[i],pp[i]=parts.update(dt)
    a.write('   Kintetic: '+ repr(kk[i]/(n**2)) +'        Potential: '+ repr(pp[i]/(n**2)) + '       total: '+ repr((kk[i]-pp[i])/(n**2))+ '\n') 
    
a.close()
    
# build gif
with imageio.get_writer('RK4.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)
    